In [1]:
##Cormac McHale

In [2]:
##Training the Neural Network

In [3]:
import gzip
import sklearn.preprocessing as pre
import keras as kr
import numpy as np
from keras.models import load_model

Using TensorFlow backend.


In [4]:
# Start a neural network, building it by layers.
model = kr.models.Sequential()

# Add a hidden layer with 1000 neurons and an input layer with 784.
model.add(kr.layers.Dense(units=1000, activation='relu', input_dim=784))
#model.add(kr.layers.Dense(units=100, activation='sigmoid'))
model.add(kr.layers.Dense(units=1000, activation='relu'))
# Add a 10 neuron output layer for each digit
model.add(kr.layers.Dense(units=10, activation='softmax'))

# Build the graph.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
##file reading and formatting
imagesFile  = open('MNIST/train-images.idx3-ubyte', 'rb')
labelsFile = open('MNIST/train-labels.idx1-ubyte', 'rb')
images = imagesFile.read()
labels = labelsFile.read()
#getting the file info ready to train the model
images =  (~np.array(list(images[16:])).reshape(60000, 28, 28).astype(np.uint8)-255)
labels =  np.array(list(labels[ 8:])).astype(np.uint8)

In [6]:
inputs = images.reshape(60000, 784)
#formatting the data to see if we can acheieve better results
inputs[inputs>0] = 1
inputs[inputs<1] = 0 

In [7]:
#label the output layer
encoder = pre.LabelBinarizer()
encoder.fit(labels)
outputs = encoder.transform(labels)
#just for looking
#print(inputs[0])
#print (inputs[0].size)

In [8]:
#view of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_3 (Dense)              (None, 10)                10010     
Total params: 1,796,010
Trainable params: 1,796,010
Non-trainable params: 0
_________________________________________________________________


In [9]:
##start training the neural network
model.fit(inputs, outputs, epochs=5, batch_size=150)

Epoch 1/5
60000/60000 [==============================] - 17s 283us/step - loss: 0.2133 - accuracy: 0.9345
Epoch 2/5
60000/60000 [==============================] - 16s 268us/step - loss: 0.0914 - accuracy: 0.9714
Epoch 3/5
60000/60000 [==============================] - 17s 286us/step - loss: 0.0613 - accuracy: 0.9799
Epoch 4/5
60000/60000 [==============================] - 18s 307us/step - loss: 0.0450 - accuracy: 0.9854
Epoch 5/5
60000/60000 [==============================] - 17s 281us/step - loss: 0.0366 - accuracy: 0.9875


In [10]:
##need to then use the test images on the Neural Network...
imagesTest  = open('MNIST/t10k-images.idx3-ubyte', 'rb')
labelsTest = open('MNIST/t10k-labels.idx1-ubyte', 'rb')
testImages = imagesTest.read()
testLabels = labelsTest.read()

In [11]:
#checking the file
#type(testImages)

In [12]:
#checking the data
#testImages[0:4]

In [13]:
#prepare the test images for prediction
testImages = (~np.array(list(testImages[16:])).reshape(10000, 28, 28).astype(np.uint8)-255)
testLabels =  np.array(list(testLabels[ 8:])).astype(np.uint8)
testInputs = testImages.reshape(10000, 784)
testInputs[testInputs>0] = 1
testInputs[testInputs<1] = 0 
encoder = pre.LabelBinarizer()
encoder.fit(testLabels)
testOutputs = encoder.transform(testLabels)

In [14]:
# Have the network predict the classes of the test inputs.
predictions = model.predict(testInputs)
predictions = encoder.inverse_transform(predictions)
##Comparison
predictions == encoder.inverse_transform(testOutputs)
#check how many it guessed correctly
(predictions == encoder.inverse_transform(testOutputs)).sum()

9731

In [15]:
#save the model
model.save('savedModel/my_model.h5')
##Initial Model finished... flask Next.. will return to this later
testInputs = testImages.reshape(10000, 784)
model.predict(testInputs)

array([[7.4269922e-12, 2.6818803e-09, 5.1290057e-09, ..., 1.0000000e+00,
        9.9823969e-11, 3.3504612e-09],
       [1.4936775e-15, 1.7781139e-09, 1.0000000e+00, ..., 1.3588238e-12,
        5.2977422e-14, 2.3290352e-17],
       [1.6171672e-12, 9.9999976e-01, 3.3155359e-08, ..., 6.5276225e-08,
        1.8538969e-08, 8.2623335e-12],
       ...,
       [3.2350620e-12, 3.1524351e-08, 6.1608113e-10, ..., 5.8410416e-07,
        1.4715457e-06, 7.2424115e-07],
       [4.4579900e-09, 3.1326840e-07, 7.8655121e-12, ..., 3.1706084e-09,
        1.6326378e-05, 1.1658897e-10],
       [2.6684253e-09, 2.5851957e-10, 4.6659018e-08, ..., 7.9120712e-12,
        1.8778165e-08, 3.6307540e-11]], dtype=float32)

In [16]:
##Model complete

In [17]:
##Re-train with an extra layer

In [18]:
type(model)

keras.engine.sequential.Sequential

In [19]:
##loop through old model
#and add all the trained layers to a new model and make them un-trainable
fineTune = kr.models.Sequential()
for layer in model.layers:
    fineTune.add(layer)
for layer in fineTune.layers:
    layer.trainable = False

In [20]:
#add an extra output layer
#with sigmoid activation
#refine the guess once more
fineTune.add(kr.layers.Dense(units=10, activation='sigmoid'))

In [21]:
#compile and re-train
fineTune.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
fineTune.fit(inputs, outputs, epochs=5, batch_size=50)

Epoch 1/5
60000/60000 [==============================] - 6s 95us/step - loss: 0.3854 - accuracy: 0.9061
Epoch 2/5
60000/60000 [==============================] - 5s 85us/step - loss: 0.1490 - accuracy: 0.9852
Epoch 3/5
60000/60000 [==============================] - 5s 78us/step - loss: 0.0787 - accuracy: 0.9979
Epoch 4/5
60000/60000 [==============================] - 4s 74us/step - loss: 0.0465 - accuracy: 0.9980
Epoch 5/5
60000/60000 [==============================] - 4s 69us/step - loss: 0.0294 - accuracy: 0.9982


In [22]:
# Have the network predict the classes of the test inputs.
predictions = fineTune.predict(testInputs)
predictions = encoder.inverse_transform(predictions)
##Comparison
predictions == encoder.inverse_transform(testOutputs)
#check how many it guesses correctly
(predictions == encoder.inverse_transform(testOutputs)).sum()

9731

In [23]:
#Results
#save new model
fineTune.save('savedModel/freshlyTrained.h5')
#test in client